The goal of this project:
1. Scrape lyrics of two artist (selena gomez and Ed sheeren) form the web 
2. Train and Predict Artist based on the lyrics using Multinomial Naive bayes

In [1]:
import pprint

#Library to download song lyrics
import requests
import re
from bs4 import BeautifulSoup

#lib to preprocess lyrics
import nltk
from nltk.tokenize import TreebankWordTokenizer 
from nltk.stem import WordNetLemmatizer
from neattext import clean_text
from nltk.corpus import stopwords

#lib for classification and prediction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB






Download lyrics of Selena Gomez and Ed Sheren

In [2]:
#Use the requests module to download the page 
URL_sel = requests.get('https://www.lyrics.com/artist/Selena-Gomez/1063097').text
URL_ed = requests.get('https://www.lyrics.com/artist/Ed-Sheeran').text

In [3]:
#Extract the links using regular expression
pattern = "lyric/\d+/Selena\+Gomez/[^\"]+"
Selena_lyrics=re.findall(pattern, URL_sel, re.IGNORECASE) #findall to get all the query pattern
pprint.pprint(Selena_lyrics)

['lyric/36037902/Selena+Gomez/I+Can%27t+Get+Enough',
 'lyric/35949460/Selena+Gomez/Anxiety',
 'lyric/36625970/Selena+Gomez/Look+at+Her+Now',
 'lyric/36628445/Selena+Gomez/Lose+You+to+Love+Me',
 'lyric/35045490/Selena+Gomez/Back+to+You',
 'lyric/35074301/Selena+Gomez/Back+to+You',
 'lyric/35567473/Selena+Gomez/Back+to+You',
 'lyric/35092444/Selena+Gomez/Revival',
 'lyric/35221589/Selena+Gomez/Wolves',
 'lyric/35227086/Selena+Gomez/Wolves',
 'lyric/34803912/Selena+Gomez/Wolves',
 'lyric/34962678/Selena+Gomez/Wolves',
 'lyric/35207964/Selena+Gomez/Wolves',
 'lyric/34789202/Selena+Gomez/Wolves',
 'lyric/35263516/Selena+Gomez/Back+to+You',
 'lyric/35204927/Selena+Gomez/I+Want+You+to+Know',
 'lyric/34783152/Selena+Gomez/Come+%26+Get+It',
 'lyric/34855423/Selena+Gomez/Wolves',
 'lyric/35169419/Selena+Gomez/Wolves',
 'lyric/35477442/Selena+Gomez/Taki+Taki',
 'lyric/35074514/Selena+Gomez/Wolves',
 'lyric/34406497/Selena+Gomez/It+Ain%27t+Me',
 'lyric/33888395/Selena+Gomez/Only+You',
 'lyric/3388

In [4]:
pattern = "lyric/\d+/Ed\+Sheeran/[^\"]+"
Ed_lyrics=re.findall(pattern, URL_ed, re.IGNORECASE)
pprint.pprint(Ed_lyrics)

['lyric/36137186/Ed+Sheeran/Lay+It+All+on+Me',
 'lyric/36131004/Ed+Sheeran/Lay+It+All+on+Me',
 'lyric/36370642/Ed+Sheeran/BLOW',
 'lyric/36355896/Ed+Sheeran/Beautiful+People',
 'lyric/36370638/Ed+Sheeran/Best+Part+of+Me',
 'lyric/36234915/Ed+Sheeran/Cross+Me',
 'lyric/36156455/Ed+Sheeran/Beast+of+Burden',
 'lyric/36221045/Ed+Sheeran/I+Don%27t+Care',
 'lyric/36259016/Ed+Sheeran/Beautiful+People',
 'lyric/36259015/Ed+Sheeran/South+of+the+Border',
 'lyric/36259014/Ed+Sheeran/Cross+Me',
 'lyric/36259013/Ed+Sheeran/Take+Me+Back+to+London',
 'lyric/36259012/Ed+Sheeran/Best+Part+of+Me',
 'lyric/36259011/Ed+Sheeran/I+Don%27t+Care',
 'lyric/36259010/Ed+Sheeran/Antisocial',
 'lyric/36259009/Ed+Sheeran/Remember+the+Name',
 'lyric/36259008/Ed+Sheeran/Feels',
 'lyric/36259007/Ed+Sheeran/Put+It+All+on+Me',
 'lyric/36259006/Ed+Sheeran/Nothing+on+You',
 'lyric/36259005/Ed+Sheeran/I+Don%27t+Want+Your+Money',
 'lyric/36259004/Ed+Sheeran/1000+Nights',
 'lyric/36259003/Ed+Sheeran/Way+to+Break+My+Heart',
 

In [5]:
#Write a loop that goes through all song to add the URL link
for song in range(len(Selena_lyrics)):
    Selena_lyrics[song] = 'https://www.lyrics.com/' + Selena_lyrics[song] 
pprint.pprint(Selena_lyrics)

['https://www.lyrics.com/lyric/36037902/Selena+Gomez/I+Can%27t+Get+Enough',
 'https://www.lyrics.com/lyric/35949460/Selena+Gomez/Anxiety',
 'https://www.lyrics.com/lyric/36625970/Selena+Gomez/Look+at+Her+Now',
 'https://www.lyrics.com/lyric/36628445/Selena+Gomez/Lose+You+to+Love+Me',
 'https://www.lyrics.com/lyric/35045490/Selena+Gomez/Back+to+You',
 'https://www.lyrics.com/lyric/35074301/Selena+Gomez/Back+to+You',
 'https://www.lyrics.com/lyric/35567473/Selena+Gomez/Back+to+You',
 'https://www.lyrics.com/lyric/35092444/Selena+Gomez/Revival',
 'https://www.lyrics.com/lyric/35221589/Selena+Gomez/Wolves',
 'https://www.lyrics.com/lyric/35227086/Selena+Gomez/Wolves',
 'https://www.lyrics.com/lyric/34803912/Selena+Gomez/Wolves',
 'https://www.lyrics.com/lyric/34962678/Selena+Gomez/Wolves',
 'https://www.lyrics.com/lyric/35207964/Selena+Gomez/Wolves',
 'https://www.lyrics.com/lyric/34789202/Selena+Gomez/Wolves',
 'https://www.lyrics.com/lyric/35263516/Selena+Gomez/Back+to+You',
 'https://ww

In [6]:
for song in range(len(Ed_lyrics)):
    Ed_lyrics[song] = 'https://www.lyrics.com/' + Ed_lyrics[song] 
pprint.pprint(Ed_lyrics)

['https://www.lyrics.com/lyric/36137186/Ed+Sheeran/Lay+It+All+on+Me',
 'https://www.lyrics.com/lyric/36131004/Ed+Sheeran/Lay+It+All+on+Me',
 'https://www.lyrics.com/lyric/36370642/Ed+Sheeran/BLOW',
 'https://www.lyrics.com/lyric/36355896/Ed+Sheeran/Beautiful+People',
 'https://www.lyrics.com/lyric/36370638/Ed+Sheeran/Best+Part+of+Me',
 'https://www.lyrics.com/lyric/36234915/Ed+Sheeran/Cross+Me',
 'https://www.lyrics.com/lyric/36156455/Ed+Sheeran/Beast+of+Burden',
 'https://www.lyrics.com/lyric/36221045/Ed+Sheeran/I+Don%27t+Care',
 'https://www.lyrics.com/lyric/36259016/Ed+Sheeran/Beautiful+People',
 'https://www.lyrics.com/lyric/36259015/Ed+Sheeran/South+of+the+Border',
 'https://www.lyrics.com/lyric/36259014/Ed+Sheeran/Cross+Me',
 'https://www.lyrics.com/lyric/36259013/Ed+Sheeran/Take+Me+Back+to+London',
 'https://www.lyrics.com/lyric/36259012/Ed+Sheeran/Best+Part+of+Me',
 'https://www.lyrics.com/lyric/36259011/Ed+Sheeran/I+Don%27t+Care',
 'https://www.lyrics.com/lyric/36259010/Ed+She

In [7]:
#create a function to scrape corpus of lyrics
def lyrics_corpus(lyrics_links):
    lyrics=[]
    for link in lyrics_links:
        check=requests.head(link)
        status_link=check.status_code
        if status_link ==200:
            lyrics_text=requests.get(link).text
            lyrics_soup=BeautifulSoup(lyrics_text, 'html.parser')
            lyrics_corpus=lyrics_soup.find(attrs={"class":"lyric-body"}).text
            lyrics.append(lyrics_corpus)
        else:
            continue
    return lyrics           


In [8]:
#create corpus of Selena and Ed Sheren lyrics
selena_corpus=lyrics_corpus(Selena_lyrics)
Ed_corpus=lyrics_corpus(Ed_lyrics)


Preprocess of lyrics

In [9]:
print(len(selena_corpus))
print(len(Ed_corpus))

570
612


In [10]:
Ed_corpus=Ed_corpus[:570]  ##create balance samples
len(Ed_corpus)

570

In [11]:
#merge lyrics and create labels for the corpus
prep_corpus=selena_corpus + Ed_corpus ##corpus is X
labels= ['selena']*570 + ['Ed_sheren']*570  ##labels is y


In [12]:
#clean the lyrics text
clean_corpus=[(clean_text(elements, stopwords=True, numbers=True, special_char=True, puncts=True)).replace('\r\n', ' ').replace('\n',' ') for elements in prep_corpus]
pprint.pprint(clean_corpus)

['crazy like like lets crazy contact impact want daily breath getting deeper '
 'deeper lately like baby cause cant yeah cant yeah cant love love cant yeah '
 'cause cant leggo dice que le es suficiente wuh t mal de la mente yah cuando '
 'est solita que entre eh msica para ponerla en ambiente yeh yeah ella quiere '
 'que lo hagamo como aquella yah le busqu otro trago por si tena sed uh lo que '
 'se pone bonito se le bonito se le ve empezamo pie ahora andamo en el jet wuh '
 'vamo calentar baby t va subir bajar se quiere olvidar lo quiere recordar '
 'baby yo quiero entrar cause cant yah yeah cant yah latino gang yeah cant '
 'love love cant come on yeah cause cant cant crazy like like lets crazy '
 'contact impact want daily breath getting deeper deeper lately like baby yeah '
 'uhh ones gotta know lowkey ill fine long youre tell em incredible '
 'unforgettable body leggo leggo tainy come on cause cant benny benny blanco '
 'yeah cant j balvin man selena yeah cant love love cant yeah

In [13]:
##clean lyrics text (2)
Lyric_tokens=[]
tokenizer = TreebankWordTokenizer()
lemmatizer = WordNetLemmatizer()

for doc in clean_corpus:
    tokens = tokenizer.tokenize(text=doc)
    clean_doc = " ".join(lemmatizer.lemmatize(token) for token in tokens)
    Lyric_tokens.append(clean_doc)

In [14]:
pprint.pprint(Lyric_tokens)

['crazy like like let crazy contact impact want daily breath getting deeper '
 'deeper lately like baby cause cant yeah cant yeah cant love love cant yeah '
 'cause cant leggo dice que le e suficiente wuh t mal de la mente yah cuando '
 'est solita que entre eh msica para ponerla en ambiente yeh yeah ella quiere '
 'que lo hagamo como aquella yah le busqu otro trago por si tena sed uh lo que '
 'se pone bonito se le bonito se le ve empezamo pie ahora andamo en el jet wuh '
 'vamo calentar baby t va subir bajar se quiere olvidar lo quiere recordar '
 'baby yo quiero entrar cause cant yah yeah cant yah latino gang yeah cant '
 'love love cant come on yeah cause cant cant crazy like like let crazy '
 'contact impact want daily breath getting deeper deeper lately like baby yeah '
 'uhh one got ta know lowkey ill fine long youre tell em incredible '
 'unforgettable body leggo leggo tainy come on cause cant benny benny blanco '
 'yeah cant j balvin man selena yeah cant love love cant yeah ca

Training and Evalaution of model 

In [15]:
###split data to test and Train
X_train, X_test, y_train, y_test = train_test_split(Lyric_tokens, labels, random_state=10) 

In [16]:
#vectorization of test and train data
STOPWORDS = stopwords.words('english')

vectorizer=TfidfVectorizer(stop_words=STOPWORDS)# initialize the vectorizer

X_train_vex=vectorizer.fit_transform(X_train)  ##train

X_test_vex=vectorizer.transform(X_test)  ##testn

In [17]:
#Train model
model = MultinomialNB()  #initilaize the model

model.fit(X_train_vex, y_train)  #fit model

MultinomialNB()

In [18]:
#Evalute the model
print(model.score(X_train_vex, y_train))

print(model.score(X_test_vex, y_test))

0.9766081871345029
0.9438596491228071
